In [2]:
# Import dependencies
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import classification_report
#Setting a nice style always make visualizations stand out more ;)
sns.set_style('whitegrid')
palette = sns.color_palette("crest")[-1:0:-4]

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('dataset.csv')

df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [4]:
df.drop(['RowNumber', 'CustomerId'], axis=1, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Surname             10000 non-null  object 
 1   CreditScore         10000 non-null  int64  
 2   Geography           10000 non-null  object 
 3   Gender              10000 non-null  object 
 4   Age                 10000 non-null  int64  
 5   Tenure              10000 non-null  int64  
 6   Balance             10000 non-null  float64
 7   NumOfProducts       10000 non-null  int64  
 8   HasCrCard           10000 non-null  int64  
 9   IsActiveMember      10000 non-null  int64  
 10  EstimatedSalary     10000 non-null  float64
 11  Exited              10000 non-null  int64  
 12  Complain            10000 non-null  int64  
 13  Satisfaction Score  10000 non-null  int64  
 14  Card Type           10000 non-null  object 
 15  Point Earned        10000 non-null  int64  
dtypes: fl

In [6]:
df.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203800,0.204400,3.013800,606.515100
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402842,0.403283,1.405919,225.924839
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,1.000000,119.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,2.000000,410.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,0.000000,3.000000,605.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,0.000000,4.000000,801.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,1.000000,5.000000,1000.000000


In [7]:
# Checking the distribution and unique values of all variables, we can define which ones can be converted to categorical
df.select_dtypes(include=['int', 'float']).nunique()

CreditScore            460
Age                     70
Tenure                  11
Balance               6382
NumOfProducts            4
HasCrCard                2
IsActiveMember           2
EstimatedSalary       9999
Exited                   2
Complain                 2
Satisfaction Score       5
Point Earned           785
dtype: int64

In [8]:
# Since HasCrCard, IsActiveMember, Exited and Complain are yes/no values, we'll change their type to categorical

df[['HasCrCard', 'IsActiveMember', 'Complain']] = df[['HasCrCard', 'IsActiveMember', 'Complain']].astype('str')

In [9]:
#Since NumOfProducts variable has categories with low frequencies, let's group them together based on this logic:

#"Does the customer has a product, or more than one?"
df['monoproduct'] = df['NumOfProducts'].apply(lambda x: 'yes' if x > 1 else 'no')

In [10]:
df.drop('NumOfProducts', axis=1, inplace=True)

In [11]:
bank = df[['Geography', 'Gender', 'IsActiveMember','monoproduct', 'Age', 'Balance', 'Exited']]

In [12]:
bank = pd.get_dummies(bank,drop_first=True)

In [13]:
X, y = bank.loc[:,bank.columns!='Exited'],bank['Exited'] 

In [14]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 17)

In [15]:
# Fit Decision Tree classifier
tree = DecisionTreeClassifier(random_state=17)

tree_params = {  
                 'criterion' : ['gini', 'entropy'],
                 'max_depth' : [2,3,4,5,6,7,8,9,10],
                 'min_samples_split' : [2,5,10,15,20,25],
                 'min_samples_leaf' : [2,4,6,8,10]
              }
tree_rs = RandomizedSearchCV(tree, tree_params, cv=8, random_state=17)

tree_rs.fit(X_train, y_train)

RandomizedSearchCV(cv=8, estimator=DecisionTreeClassifier(random_state=17),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [2, 3, 4, 5, 6, 7, 8, 9,
                                                      10],
                                        'min_samples_leaf': [2, 4, 6, 8, 10],
                                        'min_samples_split': [2, 5, 10, 15, 20,
                                                              25]},
                   random_state=17)

In [16]:
# Fit Random Forest classifier


forest = RandomForestClassifier(random_state=17)

forest_params = { 
               'n_estimators' : [50,100, 200, 400],
               'criterion' : ['gini', 'entropy'],
               'max_depth' : [2,3,4,5,6,7,8,9,10],
                 'min_samples_split' : [2,5,10,15,20,25],
                 'min_samples_leaf' : [2,4,6,8,10]
             }
forest_rs = RandomizedSearchCV(forest, forest_params, cv=8, random_state=17)
forest_rs.fit(X_train, y_train)

RandomizedSearchCV(cv=8, estimator=RandomForestClassifier(random_state=17),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [2, 3, 4, 5, 6, 7, 8, 9,
                                                      10],
                                        'min_samples_leaf': [2, 4, 6, 8, 10],
                                        'min_samples_split': [2, 5, 10, 15, 20,
                                                              25],
                                        'n_estimators': [50, 100, 200, 400]},
                   random_state=17)

In [17]:
# Fit XGBoost classifier

xgb_params = {'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.3, 0.5, 0.8],
              'max_depth': [2,3,4,5,6,7,8,9,10],
              'min_child_weight': [1, 3, 5, 7, 9],
              'subsample': [0.1, 0.3, 0.5, 0.7, 0.9],
              'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 0.9],
              'n_estimators': [50,100, 200, 400],
              'gamma': [0.1, 0.3, 0.5, 0.7, 0.9],
              'reg_alpha': [0.1, 0.3, 0.5, 0.7, 0.9],
              'reg_lambda': [0.1, 0.3, 0.5, 0.7, 0.9],
              'scale_pos_weight':[4]
             }

xgb = XGBClassifier(random_state=17)
xgb_rs = RandomizedSearchCV(xgb,xgb_params, cv=8, random_state=17)
xgb_rs.fit(X_train, y_train)

RandomizedSearchCV(cv=8,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                   param_distributions={'colsample_bytree': [0.1, 0.3, 0.5, 0.7,
                                                             0.9],
                                        'gamma': [0.1, 0.3, 0.5, 0.7, 0.9],
                                        'learning_rate': [0.01, 0.03, 0.05, 0.1,
                                                          0.3, 0.5, 0.8],
                                        'max_depth': [2, 3, 4, 5, 6, 7, 8, 9,
                                                      10],
                                        'min_child_weight': [1, 3, 5, 7, 9],
                                        'n_estimators': [50, 100, 200, 400],
                                        'reg_alpha': [0.1, 0.3, 0.5, 0.7, 0.9],
                                        'reg_lambda': [0.1, 0.3, 0.5, 0.7, 0.9],
                                        'scale_pos_weight': [4],
                                        'subsample': [0.1, 0.3, 0.5, 0.7, 0.9]},
                   random_state=17)

In [18]:
print(classification_report(y_train,  tree_rs.predict(X_train)))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      5576
           1       0.73      0.48      0.58      1424

    accuracy                           0.86      7000
   macro avg       0.80      0.72      0.75      7000
weighted avg       0.85      0.86      0.85      7000



In [19]:
print(classification_report(y_test,  tree_rs.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90      2386
           1       0.64      0.41      0.50       614

    accuracy                           0.83      3000
   macro avg       0.75      0.67      0.70      3000
weighted avg       0.82      0.83      0.82      3000



In [20]:
# Random Forest Classifier
print(classification_report(y_train,  forest_rs.predict(X_train)))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92      5576
           1       0.80      0.42      0.55      1424

    accuracy                           0.86      7000
   macro avg       0.84      0.70      0.74      7000
weighted avg       0.86      0.86      0.84      7000



In [21]:
print(classification_report(y_test,  forest_rs.predict(X_test)))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90      2386
           1       0.70      0.34      0.45       614

    accuracy                           0.83      3000
   macro avg       0.77      0.65      0.68      3000
weighted avg       0.82      0.83      0.81      3000



In [22]:
# Xtreme Gradient Boosting
print(classification_report(y_train,  xgb_rs.predict(X_train)))

              precision    recall  f1-score   support

           0       0.94      0.83      0.88      5576
           1       0.54      0.79      0.64      1424

    accuracy                           0.82      7000
   macro avg       0.74      0.81      0.76      7000
weighted avg       0.86      0.82      0.83      7000



In [23]:
print(classification_report(y_test,  xgb_rs.predict(X_test)))

              precision    recall  f1-score   support

           0       0.90      0.81      0.85      2386
           1       0.47      0.66      0.55       614

    accuracy                           0.78      3000
   macro avg       0.69      0.74      0.70      3000
weighted avg       0.81      0.78      0.79      3000

